In [1]:
import re 

In [2]:
class TSP:
    def __read (self, file):  
        f = open('./berlin52.tsp', 'r')
        match = '^[0-9].*'
        points = []
        
        for line in f:
            is_point = re.search(match, line)

            if is_point:
                x,y = map(float, line.split()[1:])
                points.append((x,y))
    
        return(points)
    
    def __init__(self, file):
        self.points = self.__read(file)

In [3]:
files = ['berlin52.tsp', 'ch150.tsp', 'd198.tsp', 'eil101.tsp']
berlin = TSP('berlin52.tsp')